In [1]:
import numpy as np
import xroms
from glob import glob
import matplotlib.pyplot as plt
import gc
import xarray as xr
import pandas as pd
import xrft as xrft
import warnings
import sys
import xrft
# import seaborn as sns
sys.path.append("/homes/metogra/iufarias/FeedbackSubmeso/useful/")
import romspickle

from dask.diagnostics import ProgressBar
import dask
# dask.config.set(**{'array.slicing.split_large_chunks': True})
from dask.distributed import Client, LocalCluster

import scipy.integrate as integ
warnings.filterwarnings("ignore")




In [2]:
# client = Client(n_workers=90)
# client = Client(cluster)

client = Client(n_workers=15,processes=True)
client

2024-09-06 19:54:41,440 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-hu5ym1bw', purging
2024-09-06 19:54:41,440 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-3f50_gdj', purging
2024-09-06 19:54:41,441 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-2f67vz2t', purging
2024-09-06 19:54:41,441 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-8p05xs66', purging
2024-09-06 19:54:41,441 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-hd53zstl', purging
2024-09-06 19:54:41,442 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-w07vqic7', purging
2024-09-06 19:54:41,442 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-tojdmocl', purging

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 15
Total threads: 135,Total memory: 502.98 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39415,Workers: 15
Dashboard: http://127.0.0.1:8787/status,Total threads: 135
Started: Just now,Total memory: 502.98 GiB
Comm: tcp://127.0.0.1:45467,Total threads: 9
Dashboard: http://127.0.0.1:45175/status,Memory: 33.53 GiB
Nanny: tcp://127.0.0.1:41725,


2024-09-06 20:58:22,297 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2024-09-06 21:20:51,140 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 15
Total threads: 135,Total memory: 502.98 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39415,Workers: 15
Dashboard: http://127.0.0.1:8787/status,Total threads: 135
Started: Just now,Total memory: 502.98 GiB
Comm: tcp://127.0.0.1:45467,Total threads: 9
Dashboard: http://127.0.0.1:45175/status,Memory: 33.53 GiB
Nanny: tcp://127.0.0.1:41725,


## Reading CROCO (3D)

In [4]:
# chunks={'time_counter':10, 'x_rho':50, 'y_rho':190,'s_rho':80,'s_w':80}
# chunks={'time_counter':1,'x_rho':1000,"x_u":1000,"x_v":1000,
#                         'y_rho':1520,"y_u":1520,"y_v":1520,'s_rho':80,'s_w':80}

chunks={'time_counter':1,'x_rho':100,"x_u":100,"x_v":100,
                        'y_rho':152,"y_u":152,"y_v":152,'s_rho':80,'s_w':80}


path='/data/pacific/lrenault/SASI/CROCO/'

# varf=xr.open_mfdataset(path + 'FULL/SASI*2012-01*.nc',data_vars='minimal',chunks=chunks)
# vart=xr.open_mfdataset(path + 'SMTH/SASI*2012-01*.nc',data_vars='minimal',chunks=chunks)

varf=xr.open_mfdataset(path + 'FULL/SASI*2012*.nc',data_vars='minimal',chunks=chunks)
vart=xr.open_mfdataset(path + 'SMTH/SASI*2012*.nc',data_vars='minimal',chunks=chunks)



In [5]:
if varf.time_counter.shape[0]>150:
    month='total'
else:
    month='{0:02d}'.format(pd.DatetimeIndex(varf.time_counter)[0].month)
print(month)

total


Converting to xroms

In [6]:
varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
df,gf=xroms.roms_dataset(varx,Vtransform=varx.Vtransform.data)

vary = vart.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
dt,gt=xroms.roms_dataset(vary,Vtransform=vary.Vtransform.data)

2024-09-06 19:59:16,971 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)


In [7]:

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 22})

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Lucida Grande']

In [8]:
timer_h=pd.DatetimeIndex(dt.ocean_time)
timer_d=pd.DatetimeIndex(dt.ocean_time.groupby("ocean_time.dayofyear").mean())

#### Spectral parameters

In [9]:
dx=0.5
nf=2
wdws='hann' #'flattop','hann'
wdw_cor=True
scl='density'
g=9.8

In [10]:
# z_interp=np.linspace(-5,-50, 10)
z_interp2=np.linspace(-1,-70, 30)

fnum=240

### $N^2_r$
JAN=2.275e-06
FEB=4.178e-06
MAR=2.116e-05
APR=2.863e-05
MAY=1.790e-05
JUN=5.648e-05
AVG=2.175e-05

In [13]:
# %%time

# N2=[]
# for ixx in range(rho_full.shape[0]):
#     N2.append(df.xroms.N2[:,:,120:-120,120:800][ixx,-2].compute().mean())
# np.mean(N2)

2023-06-10 13:26:12,603 - distributed.worker_memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 5.06 GiB -- Worker memory limit: 6.29 GiB
2023-06-10 13:26:13,145 - distributed.worker_memory - WARNING - Worker is at 31% memory usage. Resuming worker. Process memory: 1.97 GiB -- Worker memory limit: 6.29 GiB


CPU times: user 4min 48s, sys: 1min 41s, total: 6min 29s
Wall time: 19min 41s


### $N^2_r$ (FULL)

In [17]:
rho_full=df.rho

In [18]:
z=rho_full.z_rho0.compute()

In [28]:
# N2z=xr.DataArray(coords=(range(df.rho.shape[0]), range(z_interp.shape[0]),
#                          range(df.rho[:,:,120:-120,120:800].shape[2]),
#                          range(df.rho[:,:,120:-120,120:800].shape[3])),
#             dims=['ocean_time','z_rho_psi','eta_rho','xi_rho'])

N2z_2=xr.DataArray(coords=(df.rho['ocean_time'],z_interp2),
            dims=['ocean_time','z_rho_psi'])

In [29]:
%%time
# 'coarse z'
# dz=np.diff(z_interp).mean()
# for ix in range(df.rho.shape[0]):
# # for ix in range(2):

#     rho_interp_full=rho_full[:,:,120:-120,120:800][ix].xroms.isoslice(z_interp).compute()
#     b_interp_full=xroms.buoyancy(rho_interp_full)
#     N2_interp_full=np.gradient(b_interp_full,axis=0)/dz
    
#     N2z[ix,:,:,:]=N2_interp_full
    
'fine z'
dz=np.diff(z_interp2).mean()
for ix in range(df.rho.shape[0]):
# for ix in range(2):

    rho_interp_full=rho_full[:,:,120:-120,120:800][ix].xroms.isoslice(z_interp2).compute()
    b_interp_full=xroms.buoyancy(rho_interp_full)
    N2_interp_full=xr.apply_ufunc(np.gradient, b_interp_full, kwargs={'axis': 0})/dz

    N2z_2[ix,:,]=N2_interp_full.mean(['eta_rho','xi_rho'])

CPU times: user 25min 58s, sys: 12min 42s, total: 38min 41s
Wall time: 1h 21min 26s


In [29]:
N2z_mean=N2z.mean(['eta_rho','xi_rho'])

In [34]:
%%time
# N2z_mean.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/N2z_mean_full_{0}.nc'.format(month))
N2z_2.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/N2z_mean_full_fine_z_total.nc')

CPU times: user 50.7 ms, sys: 42.4 ms, total: 93.1 ms
Wall time: 205 ms


### $N^2_r$ (SMTH)

In [38]:
rho_smth=dt.rho

In [39]:
# N2z_smth=xr.DataArray(coords=(range(dt.rho.shape[0]), range(z_interp.shape[0]),
#                          range(dt.rho[:,:,120:-120,120:800].shape[2]),
#                          range(dt.rho[:,:,120:-120,120:800].shape[3])),
#             dims=['ocean_time','z_rho_psi','eta_rho','xi_rho'])

N2z_smth_2=xr.DataArray(coords=(df.rho['ocean_time'],z_interp2),
            dims=['ocean_time','z_rho_psi'])

In [41]:
%%time
# 'coarse z'
# dz=np.diff(z_interp).mean()
# for ix in range(dt.rho.shape[0]):
# # for ix in range(2):

#     rho_interp_smth=rho_smth[:,:,120:-120,120:800][ix].xroms.isoslice(z_interp).compute()
#     b_interp_smth=xroms.buoyancy(rho_interp_smth)
#     N2_interp_smth=np.gradient(b_interp_smth,axis=0)/dz
    
#     N2z_smth[ix,:,:,:]=N2_interp_smth

'fine z'
dz=np.diff(z_interp2).mean()
for ix in range(dt.rho.shape[0]):
# for ix in range(2):

    rho_interp_smth=rho_smth[:,:,120:-120,120:800][ix].xroms.isoslice(z_interp2).compute()
    b_interp_smth=xroms.buoyancy(rho_interp_smth)
    N2_interp_smth=xr.apply_ufunc(np.gradient, b_interp_smth, kwargs={'axis': 0})/dz

    N2z_smth_2[ix,:]=N2_interp_smth.mean(['eta_rho','xi_rho'])

CPU times: user 24min 48s, sys: 12min 4s, total: 36min 52s
Wall time: 1h 16min 51s


In [19]:
%%time
N2z_smth_mean=N2z_smth.mean(['eta_rho','xi_rho'])

CPU times: user 19.3 s, sys: 23.5 s, total: 42.8 s
Wall time: 33.5 s


In [42]:
%%time
# N2z_smth_mean.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/N2z_mean_smth_{0}.nc'.format(month))
N2z_smth_2.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/N2z_mean_smth_finegrid_total.nc')

CPU times: user 24.3 ms, sys: 23.6 ms, total: 48 ms
Wall time: 94.9 ms


### EKE (FULL)

In [19]:
EKE_spec_full_zt=xr.DataArray(coords=(range(df.u.shape[0]), range(z_interp.shape[0]),range(fnum)),
            dims=['ocean_time','z_rho_psi','freq_r'])

In [20]:
%%time
u_full_psi=df.u.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]
v_full_psi=df.v.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]

for ix in range(df.u.shape[0]):


    v_interp_full=v_full_psi[ix].xroms.isoslice(z_interp).compute()
    u_interp_full=u_full_psi[ix].xroms.isoslice(z_interp).compute()


    Vf_spec=xrft.isotropic_power_spectrum(v_interp_full,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)
    Uf_spec=xrft.isotropic_power_spectrum(u_interp_full,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    EKE_spec_full_zt[ix,:,:]=0.5*(Uf_spec.data + Vf_spec.data)

CPU times: user 3h 18min 43s, sys: 6h 44min 27s, total: 10h 3min 10s
Wall time: 2h 59min 33s


In [21]:
EKE_spec_full_zt['freq_r']=Vf_spec.freq_r.data
EKE_spec_full_zt['ocean_time']=df.ocean_time.data
EKE_spec_full_zt['z_rho_psi']=z_interp

In [22]:
'/homes/metogra/iufarias/Documents/data/new_spec/Eke_full_{0}.nc'.format(month)

'/homes/metogra/iufarias/Documents/data/new_spec/Eke_full_total.nc'

In [23]:
%%time
EKE_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Eke_full_{0}.nc'.format(month))

CPU times: user 60.9 ms, sys: 33.7 ms, total: 94.6 ms
Wall time: 494 ms


### EKE (SMTH)

In [24]:
EKE_spec_smth_zt=xr.DataArray(coords=(range(dt.u.shape[0]), range(z_interp.shape[0]),range(fnum)),
            dims=['ocean_time','z_rho_psi','freq_r'])

In [25]:
%%time
u_smth_psi=dt.u.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]
v_smth_psi=dt.v.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]

for ix in range(dt.u.shape[0]):


    v_interp_smth=v_smth_psi[ix].xroms.isoslice(z_interp).compute()
    u_interp_smth=u_smth_psi[ix].xroms.isoslice(z_interp).compute()


    Vt_spec=xrft.isotropic_power_spectrum(v_interp_smth,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)
    Ut_spec=xrft.isotropic_power_spectrum(u_interp_smth,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    EKE_spec_smth_zt[ix,:,:]=0.5*(Ut_spec.data + Vt_spec.data)

CPU times: user 3h 19min 43s, sys: 6h 46min 3s, total: 10h 5min 47s
Wall time: 2h 58min 40s


In [26]:
EKE_spec_smth_zt['freq_r']=Vt_spec.freq_r.data
EKE_spec_smth_zt['ocean_time']=dt.ocean_time.data
EKE_spec_smth_zt['z_rho_psi']=z_interp

In [27]:
%%time
EKE_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Eke_smth_{0}.nc'.format(month))

CPU times: user 22.8 ms, sys: 16.6 ms, total: 39.4 ms
Wall time: 218 ms


### EPE (FULL)

new way (09/06)

In [11]:
rho_star_path='/data/pacific/iufarias/tseng_z_star/rho_star/'
rho_star_full_list=glob(rho_star_path + 'rho_star_full*');rho_star_full_list.sort()

rho_star_smth_list=glob(rho_star_path + 'rho_star_smth*');rho_star_smth_list.sort()


In [12]:
%time
rho_full_psi=df.rho[:,:,120:-120,120:800]+df.rho0
rho_smth_psi=dt.rho[:,:,120:-120,120:800]+dt.rho0

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [13]:
# N2r=2.175e-05

In [16]:
z_surf_interp=-2
zSurf=rho_full_psi.z_rho0.compute().max()
zBott=rho_full_psi.z_rho0.compute().min()
dz=10
z_interpx=np.linspace(z_surf_interp,zBott, np.int(np.abs(zBott/dz)))
print('dz={0:.1f}'.format(np.diff(z_interpx).mean()) +  '   n_z={0:.0f}'.format(np.int(np.abs(zBott/dz))))
z_interp=z_interpx[:6]

dz=-10.0   n_z=507


In [ ]:
# z_interp2=np.linspace(-1,-70, 30)


In [18]:
# EPE_spec_full_zt=xr.DataArray(coords=(range(df.rho.shape[0]), z_interp,range(fnum)),
#             dims=['ocean_time','z_rho','freq_r'])
B_spec_full_zt=[]

In [19]:
%%time
for iy in range(df.rho.shape[0]):
    rho_interp_full=rho_full_psi[iy].xroms.isoslice(z_interp).expand_dims('ocean_time').compute()
    rho_ref_full=xr.open_dataarray(rho_star_full_list[iy])[:,:6]

    b_interp_full=-g*(rho_interp_full-rho_ref_full)/1025
    # b_interp_full=xroms.buoyancy(rho_interp_full)

    Bf_spec=xrft.isotropic_power_spectrum(b_interp_full.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    B_spec_full_zt.append(Bf_spec)

2024-09-06 20:47:34,636 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-09-06 21:09:38,889 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


CPU times: user 1h 17min, sys: 2h 29min 38s, total: 3h 46min 39s
Wall time: 1h 41min 2s


In [20]:
# EPE_spec_full_zt['freq_r']=Bf_spec.freq_r.data
# EPE_spec_full_zt['ocean_time']=df.ocean_time.data
# EPE_spec_full_zt['z_rho']=z_interp

# B_spec_full_zt['freq_r']=Bf_spec.freq_r.data
# B_spec_full_zt['ocean_time']=df.ocean_time.data
# B_spec_full_zt['z_rho']=z_interp

B_spec_FULL=xr.concat(B_spec_full_zt,dim='ocean_time')

In [21]:
# EPE_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Epe_full_{0}.nc'.format(month))
# B_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/B_full_{0}.nc'.format(month))
B_spec_FULL.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/B_full_reference_{0}.nc'.format(month))

### EPE(SMTH)

In [22]:
# EPE_spec_smth_zt=xr.DataArray(coords=(range(dt.rho.shape[0]), z_interp,range(fnum)),
#             dims=['ocean_time','z_rho','freq_r'])
B_spec_smth_zt=[]

In [25]:
%%time

for iy in range(dt.rho.shape[0]):
    rho_interp_smth=rho_smth_psi[iy].xroms.isoslice(z_interp).expand_dims('ocean_time').compute()
    rho_ref_smth=xr.open_dataarray(rho_star_smth_list[iy])[:,:6]

    b_interp_smth=-g*(rho_interp_smth-rho_ref_smth)/1025
    # b_interp_smth=xroms.buoyancy(rho_interp_smth)

    Bt_spec=xrft.isotropic_power_spectrum(b_interp_smth.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    B_spec_smth_zt.append(Bt_spec)

CPU times: user 1h 16min 34s, sys: 2h 31min 39s, total: 3h 48min 14s
Wall time: 1h 38min 54s


In [26]:
# EPE_spec_smth_zt['freq_r']=Bt_spec.freq_r.data
# EPE_spec_smth_zt['ocean_time']=dt.ocean_time.data
# EPE_spec_smth_zt['z_rho']=z_interp

# B_spec_smth_zt['freq_r']=Bt_spec.freq_r.data
# B_spec_smth_zt['ocean_time']=dt.ocean_time.data
# B_spec_smth_zt['z_rho']=z_interp

B_spec_SMTH=xr.concat(B_spec_smth_zt,dim='ocean_time')

In [29]:
# EPE_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Epe_smth_{0}.nc'.format(month))
# B_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Epe_smth_{0}.nc'.format(month))
B_spec_SMTH.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/B_smth_reference_{0}.nc'.format(month))

### w'b' (FULL)

In [33]:
wb_spec_full_zt=[]

In [38]:
%%time
w_full_psi=df.w[:,:,120:-120,120:800]
rho_full_psi=df.rho[:,:,120:-120,120:800]

for iy in range(df.rho.shape[0]):
    rho_interp_full=rho_full_psi[iy].xroms.isoslice(z_interp).expand_dims('ocean_time').compute()
    rho_ref_full=xr.open_dataarray(rho_star_full_list[iy])[:,:6]

    # b_interp_full=xroms.buoyancy(rho_interp_full)
    b_interp_full=-g*(rho_interp_full-rho_ref_full)/1025

    w_interp_full=w_full_psi[iy].xroms.isoslice(z_interp).expand_dims('ocean_time').compute()


    WBf_spec=xrft.isotropic_cross_spectrum(w_interp_full.drop(['nav_lat_rho', 'nav_lon_rho']),
                                           b_interp_full.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).real

    wb_spec_full_zt.append(WBf_spec)

CPU times: user 2h 26min 42s, sys: 4h 53min 10s, total: 7h 19min 52s
Wall time: 3h 4min 23s


In [39]:
# wb_spec_full_zt['freq_r']=WBf_spec.freq_r.data
# wb_spec_full_zt['ocean_time']=df.ocean_time.data
# wb_spec_full_zt['z_rho']=z_interp


wb_spec_FULL=xr.concat(wb_spec_full_zt,dim='ocean_time')

In [40]:
# wb_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/wb_full_{0}.nc'.format(month))
wb_spec_FULL.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/wb_full_referenced_{0}.nc'.format(month))

### w'b' (SMTH)

In [41]:
wb_spec_smth_zt=[]

In [42]:
%%time
w_smth_psi=dt.w[:,:,120:-120,120:800]
rho_smth_psi=dt.rho[:,:,120:-120,120:800]

for iy in range(dt.rho.shape[0]):
    rho_interp_smth=rho_smth_psi[iy].xroms.isoslice(z_interp).expand_dims('ocean_time').compute()
    rho_ref_smth=xr.open_dataarray(rho_star_smth_list[iy])[:,:6]

    # b_interp_smth=xroms.buoyancy(rho_interp_smth)
    b_interp_smth=-g*(rho_interp_smth-rho_ref_smth)/1025
    
    w_interp_smth=w_smth_psi[iy].xroms.isoslice(z_interp).expand_dims('ocean_time').compute()


    WBt_spec=xrft.isotropic_cross_spectrum(w_interp_smth.drop(['nav_lat_rho', 'nav_lon_rho']),
                                           b_interp_smth.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).real

    wb_spec_smth_zt.append(WBt_spec)

CPU times: user 2h 28min 2s, sys: 4h 53min 17s, total: 7h 21min 20s
Wall time: 3h 6min 15s


In [43]:
# wb_spec_smth_zt['freq_r']=WBt_spec.freq_r.data
# wb_spec_smth_zt['ocean_time']=dt.ocean_time.data
# wb_spec_smth_zt['z_rho']=z_interp

wb_spec_SMTH=xr.concat(wb_spec_smth_zt,dim='ocean_time')

In [44]:
# wb_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/wb_smth_{0}.nc'.format(month))
wb_spec_SMTH.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/wb_smth_referenced_{0}.nc'.format(month))